In [202]:
import numpy as np
from copy import deepcopy
import random

In [203]:
# Tile wall index values
TILE_WALL_VERTICAL = 0
TILE_WALL_HORIZONTAL = 1
TILE_WALL_CORNER_BOTTOM_LEFT = 2
TILE_WALL_CORNER_BOTTOM_RIGHT = 3
TILE_WALL_CORNER_TOP_RIGHT = 4
TILE_WALL_CORNER_TOP_LEFT = 5
TILE_FLOOR = 6
TILE_DOOR_HORIZONTAL = 7
TILE_DOOR_VERTICAL = 8

In [204]:
#Constants for genetic algorithm parameters
POPULATION_SIZE = 100
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
NUM_ROOMS = 3 #Target number of rooms
MAP_WIDTH = 10
MAP_HEIGHT = 10

In [205]:
class parameters:
    def __init__(self, population_size=POPULATION_SIZE, max_generations=MAX_GENERATIONS, mutation_rate=MUTATION_RATE,map_width= MAP_WIDTH,map_height=MAP_HEIGHT,num_rooms=NUM_ROOMS):
        self.map_width = map_width
        self.map_height = map_height
        self.population_size = population_size
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate
        self.num_rooms = num_rooms
        self.birth_rate_per_generation = 1
        self.max_number_of_generations = 50
        self.door_positions = generate_door_positions(self.map_width,self.map_height,self.num_rooms)

In [206]:
def generate_door_positions(width,height,num_rooms):
        """
        Generate immutable door positions for the map.
        Doors are restricted to being within N-3 x N-3 to ensure they are further inside the map.
        Vertical doors are accompanied by an additional door at x-1,
        Horizontal doors are accompanied by an additional door at y-1.
        """
        positions = []
        for _ in range(num_rooms - 1):  # Generate `num_rooms - 1` doors
            while True:
                # Generate door positions further inside the map
                x = random.randint(2, width - 3)
                y = random.randint(2, height - 3)
                orientation = random.choice([TILE_DOOR_HORIZONTAL, TILE_DOOR_VERTICAL])  # Random door type

                if orientation == TILE_DOOR_VERTICAL:
                    # Ensure space for the companion door at (x-1, y)
                    if x - 1 >= 1:  # Check bounds
                        positions.append((x, y, TILE_DOOR_VERTICAL))  # Main door
                        positions.append((x - 1, y, TILE_DOOR_VERTICAL))  # Companion door
                        break
                elif orientation == TILE_DOOR_HORIZONTAL:
                    # Ensure space for the companion door at (x, y-1)
                    if y - 1 >= 1:  # Check bounds
                        positions.append((x, y, TILE_DOOR_HORIZONTAL))  # Main door
                        positions.append((x, y - 1, TILE_DOOR_HORIZONTAL))  # Companion door
                        break

        return positions

In [207]:
class LevelGenerationProblem:
    def __init__(self, params):
        self.width = params.map_width  # Map width (N)
        self.height = params.map_height  # Map height (N)
        self.num_rooms = params.num_rooms  # Number of rooms
        self.door_positions = params.door_positions


    def calculate_fitness(self, chromosome):
        """
        Fitness evaluation for a map, returning fitness and a score map.
        """
        map_with_doors = chromosome
        # Initialize fitness and create a score map same shape as the map
        fitness_score = 0
        score_map = np.zeros((self.height, self.width), dtype=int)

        # Iterate through every tile on the map
        for y in range(self.height):
            for x in range(self.width):
                tile_type = map_with_doors[y, x]

                # Skip non-evaluable tiles
                if tile_type not in [
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_VERTICAL,
                    TILE_DOOR_HORIZONTAL,
                    TILE_DOOR_VERTICAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                ]:
                    continue

                # Evaluate the single tile at (x, y)
                is_valid, tile_score = self.validate_tile(map_with_doors, x, y)

                # Aggregate the tile's contribution to the total fitness
                fitness_score += tile_score

                # Update the score map for the current tile
                score_map[y, x] = tile_score


        valid_wall_map = self.validate_connected_tiles(score_map, map_with_doors)

        # Return total fitness and the aligned score map as a 2D numpy array
        return fitness_score, score_map, valid_wall_map


    def validate_tile(self, tile_map, x, y):
        """
        Evaluate a single tile in the given map.
        Returns:
            is_valid (bool): Whether the tile contributes validly to the enclosure.
            tile_score (int): The score of the tile.
        """
        # Ensure the tile lies within valid map bounds
        if not (0 <= x < self.width and 0 <= y < self.height):
            return False, -1, {(x, y): -1}  # Invalid position outside map bounds

        tile = tile_map[y, x]  # Access the tile
        is_valid = True  # By default, assume the tile is valid
        tile_score = 0  # Tile's contribution to fitness

        # Evaluate the tile's score based on its type
        if tile in [TILE_WALL_HORIZONTAL, TILE_DOOR_HORIZONTAL]:
            is_valid, tile_score = self.validate_horizontal_tile(tile_map, x, y)
        elif tile in [TILE_WALL_VERTICAL, TILE_DOOR_VERTICAL]:
            is_valid, tile_score = self.validate_vertical_tile(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_BOTTOM_LEFT:
            is_valid, tile_score = self.validate_bottom_left_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_BOTTOM_RIGHT:
            is_valid, tile_score = self.validate_bottom_right_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_TOP_RIGHT:
            is_valid, tile_score = self.validate_top_right_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_TOP_LEFT:
            is_valid, tile_score = self.validate_top_left_corner(tile_map, x, y)
        elif tile == TILE_FLOOR:  # Assign flat score for floor tiles
            is_valid, tile_score = True, 1
        else:  # Invalid or unknown tile types
            is_valid, tile_score = False, -1

        # Return the results for this single tile
        return is_valid, tile_score



    def validate_horizontal_tile(self, tile_map, x, y):
        """
        Validate a horizontal wall or door tile by checking its left and right neighbors.
        Horizontal doors contribute additional positive score compared to walls.
        """
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_VERTICAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_VERTICAL

        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_TOP_LEFT,TILE_DOOR_HORIZONTAL]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_left and valid_right
        # Add extra score if it's a horizontal door
        base_score = 10
        #bonuses
        door_bonus = 50 if tile_map[y, x] == TILE_DOOR_HORIZONTAL else 0
        neighbor_door_bonus = 20 if left == TILE_DOOR_HORIZONTAL or right == TILE_DOOR_HORIZONTAL else 0

        score = base_score + door_bonus + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_vertical_tile(self, tile_map, x, y):
        """
        Validate a vertical wall or door tile by checking its top and bottom neighbors.
        Vertical doors contribute additional positive score compared to walls.
        """
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_HORIZONTAL
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_HORIZONTAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT,TILE_DOOR_VERTICAL]
        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT,TILE_DOOR_VERTICAL]

        is_valid = valid_above and valid_below
        # Add extra score if it's a vertical door
        base_score = 10
        # bonuses for validity
        door_bonus = 50 if tile_map[y, x] == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus = 20 if above == TILE_DOOR_VERTICAL or below == TILE_DOOR_VERTICAL else 0

        score = base_score + door_bonus + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score


    def validate_bottom_left_corner(self, tile_map, x, y):
        """Validate a bottom-left corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_HORIZONTAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_VERTICAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_VERTICAL]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_above and valid_right
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if above == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if right == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_bottom_right_corner(self, tile_map, x, y):
        """Validate a bottom-right corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_HORIZONTAL
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_VERTICAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_VERTICAL]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_above and valid_left
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if above == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if left == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_top_right_corner(self, tile_map, x, y):
        """Validate a top-right corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_HORIZONTAL
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_VERTICAL

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_VERTICAL]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_below and valid_left
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if below == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if left == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_top_left_corner(self, tile_map, x, y):
        """Validate a top-left corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_HORIZONTAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_VERTICAL

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_VERTICAL]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_below and valid_right
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if below == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if right == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_connected_tiles(self, score_map, tile_map):
        """
        Validates connected regions of tiles, ensuring walls connect correctly to doors and other walls.
        Takes into account the `tile_map` for tile-specific rules and applies:
          - Base rewards for valid walls connected to a door without forming loops.
          - Additional rewards for valid loops.
          - Penalties for walls connecting to another door.

        Inputs:
            score_map: Fitness scoring map for penalties/rewards.
            tile_map: Map containing the specific tile type at each (x, y).

        Outputs:
            valid_wall_map: Map showing which tiles are valid, penalized, or rewarded.
        """
        valid_wall_map = np.zeros((self.height, self.width), dtype=int)
        visited = set()  # Track visited tiles to avoid redundant checks

        def is_valid_tile_neighbor(tile_pos, neighbor_pos, tile_map):
            """
            Validates whether `tile_pos` and `neighbor_pos` connect correctly, considering
            both the tile types in `tile_map` and the relative position of the neighbor.
            """
            ty, tx = tile_pos
            ny, nx = neighbor_pos

            # Ensure neighbors are within bounds
            if not (0 <= nx < self.width and 0 <= ny < self.height):
                return False  # Out-of-bounds tiles are invalid neighbors

            # Tile types
            tile_type = tile_map[ty, tx]
            neighbor_type = tile_map[ny, nx]

            # Determine the relative position of the neighbor
            relative_position = (ny - ty, nx - tx)  # (row_diff, col_diff)

            # Check rules for connecting based on the current tile type
            if tile_type == TILE_WALL_VERTICAL:
                # Neighbors above or below
                if relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT,
                                             TILE_DOOR_VERTICAL]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT,
                                             TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_HORIZONTAL:
                # Neighbors to the left or right
                if relative_position == (0, -1):  # Left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_WALL_CORNER_BOTTOM_RIGHT,
                                             TILE_DOOR_HORIZONTAL]
                elif relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_BOTTOM_LEFT,
                                             TILE_DOOR_HORIZONTAL]

            elif tile_type == TILE_WALL_CORNER_TOP_LEFT:
                # Valid neighbors for top-left corner
                if relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_CORNER_TOP_RIGHT:
                # Valid neighbors for top-right corner
                if relative_position == (0, -1):  # left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_CORNER_BOTTOM_LEFT:
                # Valid neighbors for bottom-left corner
                if relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_CORNER_BOTTOM_RIGHT:
                # Valid neighbors for bottom-right corner
                if relative_position == (0, -1):  # Left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_DOOR_VERTICAL:
                # Vertical doors can connect to vertical walls or corners above and below
                if relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT]

            elif tile_type == TILE_DOOR_HORIZONTAL:
                # Horizontal doors can connect to horizontal walls or corners left and right
                if relative_position == (0, -1):  # Left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_WALL_CORNER_BOTTOM_RIGHT]
                elif relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_BOTTOM_LEFT]

            return False  # If no rules apply, the neighbor is invalid



        def is_connected_to_door(y, x, tile_map, visited, score_map, original_direction=None):
            """
            Checks if a tile is connected to a door using Depth-First Search (DFS).
            Considers tile type, avoids revisiting tiles, and checks score_map validity.
            """

            if (
                    (y, x) in visited
                    or not (0 <= x < self.width and 0 <= y < self.height)
                    or score_map[y][x] == -3
            ):
                return False

            visited.add((y, x))

            if tile_map[y][x] in [TILE_DOOR_HORIZONTAL, TILE_DOOR_VERTICAL]:
                return True

            current_tile_type = tile_map[y][x]
            neighbors = []
            if current_tile_type == TILE_WALL_VERTICAL:
                neighbors = [(y + 1, x), (y - 1, x)]  # Check up and down for vertical walls
            elif current_tile_type == TILE_WALL_HORIZONTAL:
                neighbors = [(y, x + 1), (y, x - 1)]  # Check left and right for horizontal walls
            elif current_tile_type == TILE_WALL_CORNER_TOP_LEFT:
                neighbors = [(y, x + 1), (y + 1, x)]
            elif current_tile_type == TILE_WALL_CORNER_TOP_RIGHT:
                neighbors = [(y, x - 1), (y + 1, x)]
            elif current_tile_type == TILE_WALL_CORNER_BOTTOM_LEFT:
                neighbors = [(y, x + 1), (y - 1, x)]
            elif current_tile_type == TILE_WALL_CORNER_BOTTOM_RIGHT:
                neighbors = [(y, x - 1), (y - 1, x)]
            elif current_tile_type in [TILE_DOOR_VERTICAL, TILE_DOOR_HORIZONTAL]:
                return True  # Consider doors as inherently connected regardless of neighbors
            else:  # other tile types, if any
                neighbors = []

            for ny, nx in neighbors:
                if (ny, nx) != original_direction:  # Avoid checking tile we came from
                    if is_valid_tile_neighbor(
                            (y, x), (ny, nx), tile_map
                    ) and is_connected_to_door(
                        ny, nx, tile_map, visited.copy(), score_map, (y, x)
                    ):
                        return True
            return False


        for y in range(self.height):
            for x in range(self.width):
                if score_map[y][x] != -3:  # Only check valid tiles according to the score_map
                    if is_connected_to_door(y,x, tile_map, set(), score_map):  # Pass empty set for new DFS
                        valid_wall_map[y, x] = 1

        return valid_wall_map






In [208]:
class LevelIndividual:
    """Represents an individual in the genetic algorithm."""
    def __init__(self, problem):
        self.problem = problem
        self.door_positions = problem.door_positions
        self.chromosome = self.generate_chromosome() # Chromosome is an N x N 2D tile matrix

    def generate_chromosome(self):
        # Step 1: Generate a random base chromosome
        chromosome = np.zeros((self.problem.height, self.problem.width), dtype=int)

        for y in range(self.problem.height):
            for x in range(self.problem.width):
                # Randomly assign any possible wall type
                chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                ])

        # Step 2: Add door positions into the chromosome
        for x, y, orientation in self.door_positions:
            chromosome[y, x] = orientation  # Set the door orientation at the given coordinates

        return chromosome


    def mutate(self, mutation_rate):
        """
        Mutate the chromosome by randomly changing tiles, excluding doors.
        """
        for _ in range(int(self.problem.width * self.problem.height * mutation_rate)):
            x = random.randint(0, self.problem.width - 1)
            y = random.randint(0, self.problem.height - 1)

            # Ensure doors remain immutable
            if (x, y) not in [(door[0], door[1]) for door in self.door_positions]:
                self.chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT
                ])

    def crossover(self, other_parent):
        """
        Perform crossover with another parent.
        Mix parts of the two maps.
        """
        child1 = deepcopy(self)
        child2 = deepcopy(other_parent)

        # Single-point crossover on the 2D matrix
        split_row = random.randint(0, self.problem.height - 1)

        child1.chromosome[:split_row, :] = self.chromosome[:split_row, :]
        child1.chromosome[split_row:, :] = other_parent.chromosome[split_row:, :]

        child2.chromosome[:split_row, :] = other_parent.chromosome[:split_row, :]
        child2.chromosome[split_row:, :] = self.chromosome[split_row:, :]

        return child1, child2


    def generate_display_map(self):
        """
        Generates a scored map from a chromosome, highlighting positive scores and
        using spaces for non-positive scores.

        Args:
            chromosome: The chromosome representing the map.

        Returns:
            str: Human-readable formatted display map as a string.
        """
        # Decode the map and calculate fitness and scores
        tile_map = self.chromosome
        fitness, score_map, _= self.problem.calculate_fitness(tile_map)

        # Create a display map based on scores (positive scores retain the original tile)
        scored_map = np.full_like(tile_map, " ", dtype=object)
        for y in range(self.problem.height):
            for x in range(self.problem.width):
                if score_map[y, x] > 0:  # Correctly check score_map with (y, x) indexing
                    scored_map[y, x] = tile_map[y, x]  # Assign tile for positive scores
                else:
                    scored_map[y, x] = " "  # Non-positive scores are blank spaces

        # Display logic to make it human-readable
        vertical = ' | '
        horizontal = '---'
        corner_bottom_left = ' |_'
        corner_bottom_right = '_| '
        corner_top_right = '‾| '
        corner_top_left = ' |‾'

        door_horizontal = 'HD'
        door_vertical = 'VD'

        output = ""
        for row in scored_map:
            formatted_row = []
            for tile in row:
                if tile == 0:  # Vertical wall
                    formatted_row.append(vertical.ljust(3))
                elif tile == 1:  # Horizontal wall
                    formatted_row.append(horizontal.ljust(3))
                elif tile == 2:  # Corner bottom-left
                    formatted_row.append(corner_bottom_left.ljust(3))
                elif tile == 3:  # Corner bottom-right
                    formatted_row.append(corner_bottom_right.ljust(3))
                elif tile == 4:  # Corner top-right
                    formatted_row.append(corner_top_right.ljust(3))
                elif tile == 5:  # Corner top-left
                    formatted_row.append(corner_top_left.ljust(3))
                elif tile == ' ':  # Empty space for non-positive scores
                    formatted_row.append('   ')
                elif tile == 7:  # Horizontal door
                    formatted_row.append(door_horizontal.ljust(3))
                elif tile == 8:  # Vertical door
                    formatted_row.append(door_vertical.ljust(3))
                else:  # Default case for unmapped tiles
                    formatted_row.append(str(tile).ljust(3))
            output += "[" + "".join(formatted_row) + "]\n"

        return output

In [209]:
params = parameters()

In [210]:
problem = LevelGenerationProblem(params)
# View the generated door positions
print("Door positions:", problem.door_positions)


Door positions: [(6, 7, 8), (5, 7, 8), (6, 4, 8), (5, 4, 8)]


In [211]:
ind1 = LevelIndividual(problem)

In [212]:
    # Decode the individual's chromosome into a map
decoded_map = ind1.chromosome
print("Decoded Map:")
print(decoded_map)


Decoded Map:
[[2 2 0 2 5 0 2 3 0 2]
 [5 0 2 1 0 4 1 4 2 1]
 [1 4 0 5 3 2 4 3 5 2]
 [1 2 4 5 0 4 1 1 1 3]
 [4 2 0 2 1 8 8 1 1 3]
 [1 4 5 2 1 0 0 1 1 4]
 [3 2 2 5 3 5 5 4 0 4]
 [4 3 5 3 0 8 8 2 5 2]
 [3 5 0 1 3 5 5 1 1 1]
 [0 0 5 4 2 3 0 3 5 0]]


In [213]:
# Calculate fitness
fitness, scored_map, validated_wall_map = problem.calculate_fitness(ind1.chromosome)
print("Fitness:", fitness)
print("Scored Map:", scored_map)


Fitness: -35
Scored Map: [[-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 -3 10 -3 10 -3 -3 10 10 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 10 10 -3]
 [-3 -3 -3 10 -3 60 -3 -3 10 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 10 -3]
 [-3 -3 -3 -3 -3 -3 30 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 10 -3 10 10 10 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]]


In [214]:
scored_map = ind1.generate_display_map()
print(scored_map)


[                              ]
[       |_    |       ‾|  |_   ]
[                              ]
[                     ------   ]
[          |_   VD       ---   ]
[                        ---   ]
[                   |‾         ]
[                              ]
[            _|     |‾------   ]
[                              ]



In [215]:
print("Valid wall Map:", validated_wall_map)

Valid wall Map: [[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [216]:
def choose_parents(population):
    # Tournament selection
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x.cost)

In [217]:
# %%
def run_genetic(prob, params):
    # Read Variables
    population_size = params.population_size
    rate_of_gene_mutation = params.mutation_rate
    # cost_function = prob.calculate_fitness()
    number_of_children_per_generation = int(params.birth_rate_per_generation * population_size)
    max_number_of_generations = params.max_number_of_generations

    # Create Our Population
    population = []
    best_solution = LevelIndividual(prob)  # Use LevelIndividual
    best_solution.cost = -100000  # Initialize with a very low cost

    for i in range(population_size):
        new_individual = LevelIndividual(prob)  # updated individual type
        new_individual.cost, _, _ = prob.calculate_fitness(new_individual.chromosome)  # removed score map
        if new_individual.cost > best_solution.cost:
            best_solution = deepcopy(new_individual)
        population.append(new_individual)

    # Start Loop
    for i in range(max_number_of_generations):
        # Start generation loop
        children = []
        while (len(children) < number_of_children_per_generation):
            # choose Parents
            parent1 = choose_parents(population)
            parent2 = choose_parents(population)

            # Create children
            child1, child2 = parent1.crossover(parent2)
            child1.mutate(rate_of_gene_mutation)
            child2.mutate(rate_of_gene_mutation)

            child1.cost, _ , _= prob.calculate_fitness(child1.chromosome)
            child2.cost, _ , _= prob.calculate_fitness(child2.chromosome)

            # add children to population
            children.append(child1)
            children.append(child2)

        # add children
        population += children

        # sort population
        population = sorted(population, key=lambda x: x.cost, reverse=True)  # Sort in descending order

        # cull population
        population = population[:population_size]

        # check solution
        if population[0].cost > best_solution.cost:
            best_solution = deepcopy(population[0])

        print(f"Generation {i}: Best solution cost = '{best_solution.cost}'")

    print( "Best solution cost = '{best_solution.cost}'", best_solution.cost)
    return (population, best_solution)


In [218]:

params1 = parameters()
problem1 = LevelGenerationProblem(params1)

In [219]:
pop, best = run_genetic(problem1,params1)

Generation 0: Best solution cost = '331'
Generation 1: Best solution cost = '377'
Generation 2: Best solution cost = '423'
Generation 3: Best solution cost = '430'
Generation 4: Best solution cost = '469'
Generation 5: Best solution cost = '469'
Generation 6: Best solution cost = '515'
Generation 7: Best solution cost = '515'
Generation 8: Best solution cost = '547'
Generation 9: Best solution cost = '547'
Generation 10: Best solution cost = '547'
Generation 11: Best solution cost = '547'
Generation 12: Best solution cost = '554'
Generation 13: Best solution cost = '554'
Generation 14: Best solution cost = '567'
Generation 15: Best solution cost = '606'
Generation 16: Best solution cost = '606'
Generation 17: Best solution cost = '619'
Generation 18: Best solution cost = '632'
Generation 19: Best solution cost = '645'
Generation 20: Best solution cost = '645'
Generation 21: Best solution cost = '645'
Generation 22: Best solution cost = '645'
Generation 23: Best solution cost = '658'
Ge

In [220]:
fitness, scored_map, validated_wall_map = problem.calculate_fitness(best.chromosome)
print("Fitness:", fitness)
print("Scored Map:", scored_map)

Fitness: 749
Scored Map: [[-3 10 -3 -3 -3 10 10 10 10 10]
 [-3 -3 -3 10 -3 30 60 30 10 10]
 [-3 -3 10 10 10 30 60 30 10 -3]
 [10 10 -3 -3 -3 -3 -3 -3 -3 -3]
 [10 10 30 30 10 10 10 -3 -3 -3]
 [-3 -3 60 60 -3 -3 -3 -3 10 10]
 [10 10 30 30 10 -3 10 -3 -3 -3]
 [10 10 -3 -3 -3 -3 10 -3 -3 -3]
 [10 10 -3 10 10 10 -3 10 10 10]
 [-3 -3 10 -3 -3 10 10 10 -3 -3]]


In [221]:
decoded_map = best.chromosome
print("Decoded Map:")
print(decoded_map)

Decoded Map:
[[1 1 4 4 1 5 1 1 1 4]
 [0 3 4 0 2 2 7 1 4 0]
 [5 2 3 2 1 1 7 1 3 3]
 [5 4 3 4 0 0 5 5 1 2]
 [2 3 5 4 5 1 1 3 2 4]
 [3 1 8 8 2 5 3 1 1 3]
 [5 4 2 3 5 4 5 1 5 0]
 [0 0 1 0 0 4 0 4 2 0]
 [2 3 3 2 1 3 3 0 5 4]
 [2 5 1 4 2 1 1 3 0 0]]


In [222]:
display_map = best.generate_display_map()
print(display_map)

[   ---          |‾---------‾| ]
[          |     |_HD ---‾|  | ]
[      _|  |_------HD ---_|    ]
[ |‾‾|                         ]
[ |__|  |‾‾|  |‾------         ]
[      VD VD             ---_| ]
[ |‾‾|  |__|  |‾    |‾         ]
[ |  |              |          ]
[ |__|     |_---_|     |  |‾‾| ]
[      ---      ------_|       ]



In [223]:
print("Valid wall Map:")
print(validated_wall_map)

Valid wall Map:
[[0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 1 1 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
